In [1]:
# add the prefix --proxy=http://127.0.0.1:2081 to the pip install commands inside this cell



!pip install datasets
!pip install transformers
!pip install accelerate -U
!pip install evaluate
!pip install torch torchvision torchaudio
!pip install scikit-learn -U
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00


In [1]:
!pip --proxy=http://127.0.0.1:2081 install datasets
!pip --proxy=http://127.0.0.1:2081 install transformers
!pip --proxy=http://127.0.0.1:2081 install accelerate -U
!pip --proxy=http://127.0.0.1:2081 install evaluate
!pip --proxy=http://127.0.0.1:2081 install torch torchvision torchaudio
!pip --proxy=http://127.0.0.1:2081 install scikit-learn -U
!pip --proxy=http://127.0.0.1:2081 install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.1 MB/s eta 0:00:00a 0:00:010m


In [1]:
from datasets import load_from_disk
dataset = load_from_disk('squad_v2')

/home/vpmb/Documents/python/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("squad_v2")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments



tokenizer = T5Tokenizer.from_pretrained("./t5-small/")
model = T5ForConditionalGeneration.from_pretrained("./t5-small/")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:


def tokenize_function(examples):
    for example in examples:
        model_inputs = tokenizer(f"carefully analyze the text. generate question, and then answer from key information: {example['context']}", padding="max_length", truncation=True, max_length=512)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(f"{example["question"]} {example["answers"]}", padding="max_length", truncation=True, max_length=128)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

train_dataset = dataset["train"].select(range(5000))
eval_dataset = dataset["validation"].select(range(5000))

tokenized_train_dataset = tokenize_function(train_dataset)
tokenized_eval_dataset = tokenize_function(eval_dataset)


/home/vpmb/Documents/python/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [42]:
from transformers import T5Tokenizer

# Assuming tokenizer is already defined
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def tokenize_example(example):
    # Tokenize the context and question together
    model_input = tokenizer(f"context: {example['context']}",
                            padding="max_length", truncation=True, max_length=512)

    # Tokenize the answer
    with tokenizer.as_target_tokenizer():
        # Adjust this part to properly format your answers.
        # This example assumes 'answers' is a list of answer strings.
        label = tokenizer(f"{example['question']} {example['answers']['text']}", padding="max_length", truncation=True, max_length=128)
    
    model_input["labels"] = label["input_ids"]
    return model_input

# Assuming dataset is already defined and has the correct structure
train_dataset = dataset["train"].select(range(20000))
eval_dataset = dataset["validation"].select(range(10000))

tokenized_train_dataset = [tokenize_example(example) for example in train_dataset]
tokenized_eval_dataset = [tokenize_example(example) for example in eval_dataset]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=500,
    fp16=True,
    seed=42
)

    # lr_scheduler_type="linear",
    # num_train_epochs=3,
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    # warmup_steps=500,
    # weight_decay=0.01,
    # gradient_accumulation_steps=2,


In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [44]:
trainer.train()

  7%|▋         | 501/7500 [01:22<19:19,  6.03it/s]

{'loss': 0.3102, 'learning_rate': 4.667333333333333e-05, 'epoch': 0.2}


 13%|█▎        | 1001/7500 [02:45<17:54,  6.05it/s]

{'loss': 0.3038, 'learning_rate': 4.334e-05, 'epoch': 0.4}


 20%|██        | 1501/7500 [04:07<16:32,  6.04it/s]

...

                                                   
100%|██████████| 7500/7500 [24:13<00:00,  5.81it/s]

{'eval_loss': 0.5417781472206116, 'eval_runtime': 71.1836, 'eval_samples_per_second': 140.482, 'eval_steps_per_second': 17.56, 'epoch': 3.0}


100%|██████████| 7500/7500 [24:14<00:00,  5.16it/s]

{'train_runtime': 1454.7603, 'train_samples_per_second': 41.244, 'train_steps_per_second': 5.155, 'train_loss': 0.28642506103515625, 'epoch': 3.0}


TrainOutput(global_step=7500, training_loss=0.28642506103515625, metrics={'train_runtime': 1454.7603, 'train_samples_per_second': 41.244, 'train_steps_per_second': 5.155, 'train_loss': 0.28642506103515625, 'epoch': 3.0})

In [45]:
model.save_pretrained("./t5-qg&qa_v3")


In [52]:
context = "context :  Rick and Morty is an American adult animated science fiction sitcom created by Justin Roiland and Dan Harmon for Cartoon Network's nighttime programming block Adult Swim. The series follows the misadventures of Rick Sanchez, a cynical mad scientist, and his good-hearted but fretful grandson Morty Smith, who split their time between domestic life and interdimensional adventures that take place across an infinite number of realities, often traveling to other planets and dimensions through portals and on Rick's flying saucer."
model = T5ForConditionalGeneration.from_pretrained("./t5-qg&qa_v3")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
input_ids = tokenizer(context, return_tensors="pt", max_length=128).input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/vpmb/Documents/python/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Who created Rick and Morty? ['Juice Roiland and Dan Harmon'


In [27]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

context = """The future of Google’s app store is at stake in a lawsuit by Fortnite publisher Epic Games. Epic sued Google in 2020 after a fight over in-app purchase fees, claiming the Android operating system’s Google Play store constituted an unlawful monopoly. It wants Google to make using third-party app stores, sideloaded apps, and non-Google payment processors easier — while Google says its demands would damage Android’s ability to offer a secure user experience and compete with Apple’s iOS.

The case has had a long road to court, arriving there long after a similar trial against Apple in 2021. Follow along with updates here."""

model = T5ForConditionalGeneration.from_pretrained("./t5-qg_v1")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

input_ids = tokenizer(context, return_tensors="pt").input_ids

# Adjusting generation parameters
outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Which publisher sued Google in 2020?


In [13]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

True
